# 05 — Cross-Validation (LOSO)

Leave-One-Subject-Out evaluation on DEAP (32 folds)
and session-wise on IEMOCAP (5 folds).

In [ ]:
import sys, os
os.chdir('/content/amers')
sys.path.insert(0, '.')

from pathlib import Path
import numpy as np
import torch
import matplotlib.pyplot as plt

from src.utils.config import load_config
from src.utils.seed import set_seed

DRIVE_BASE = Path('/content/drive/MyDrive/AMERS')
cfg = load_config('config/default.yaml')
set_seed(cfg.seed)

In [ ]:
# Load per-subject data
from src.data.deap_loader import DEAPLoader
from src.data.label_mapper import LabelMapper

processed_dir = str(DRIVE_BASE / 'data' / 'deap' / 'processed')
loader = DEAPLoader(processed_dir=processed_dir, label_mapper=LabelMapper())

features_by_subj = {}
labels_by_subj = {}
for s in range(1, 33):
    feats, lbls = loader.load_subject(s)
    if feats is not None:
        features_by_subj[s] = feats
        labels_by_subj[s] = lbls

subjects = sorted(features_by_subj.keys())
print(f'Loaded {len(subjects)} subjects')

In [ ]:
# LOSO CV
from src.evaluation.cross_validation import loso_cv
from src.evaluation.metrics import compute_all_metrics

# Define a simple train+eval function for EEG-only baseline
from src.models.eeg_encoder import EEGEncoder
from src.models.classifier import EncoderClassifier

def train_and_eval(train_X, train_y, test_X, test_y, cfg):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    encoder = EEGEncoder(input_dim=160, embedding_dim=128).to(device)
    model = EncoderClassifier(encoder, 128, 4).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    
    X_t = torch.as_tensor(train_X, dtype=torch.float32).to(device)
    y_t = torch.as_tensor(train_y, dtype=torch.long).to(device)
    
    model.train()
    for _ in range(20):
        logits = model(X_t)
        loss = criterion(logits, y_t)
        opt.zero_grad()
        loss.backward()
        opt.step()
    
    model.eval()
    with torch.no_grad():
        X_v = torch.as_tensor(test_X, dtype=torch.float32).to(device)
        preds = model(X_v).argmax(1).cpu().numpy()
    
    metrics = compute_all_metrics(test_y, preds)
    return preds, metrics

# Run LOSO
cv_results = loso_cv(
    subjects, features_by_subj, labels_by_subj,
    train_and_eval, cfg,
)

print(f"\nMean accuracy: {cv_results['mean_metrics']['accuracy']:.4f} ± {cv_results['std_metrics']['accuracy']:.4f}")
print(f"Mean F1 macro: {cv_results['mean_metrics']['f1_macro']:.4f} ± {cv_results['std_metrics']['f1_macro']:.4f}")

In [ ]:
# Per-fold accuracy
fold_accs = [m['accuracy'] for m in cv_results['fold_metrics']]
plt.figure(figsize=(12, 4))
plt.bar(range(1, len(fold_accs)+1), fold_accs)
plt.axhline(y=np.mean(fold_accs), color='r', linestyle='--', label=f'Mean={np.mean(fold_accs):.3f}')
plt.xlabel('Subject')
plt.ylabel('Accuracy')
plt.title('LOSO Per-Subject Accuracy')
plt.legend()
plt.show()